<a href="https://colab.research.google.com/github/CarloRomeoGitHub/ML-exercises/blob/master/ML_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#from google.colab import files
#!pip install -q kaggle


In [0]:
#uploaded = files.upload()


In [0]:
#IMPORT
import sklearn
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn.utils
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import math


In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [15]:
print(train.info(), test.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float

We can notice that test set miss a lot of entries in the column "cabin". It will be a little bit tricky to fill all missing features of test, considering that test values are less that mean values (in number) of train values.
First moment we will work only on train set.

In [16]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Sex and Embarked columns have string values.
Embarked means port of departed of specific passenger. I think that we don't really need this information now. The Fare column is embedded in Pclass.
Sex column has to be converted in int form. 
From info() and describe() we understand that Age miss some values (714 on 891).
I will use the Age column mean value: 29.

In [0]:
def preprocess(df):
  preprocess_df = df
  preprocess_df.drop(['PassengerId', 'Embarked', 'Fare', 'Ticket', 'Cabin', 'Name'], 1, inplace= True)
  preprocess_df['Age'].fillna('29', inplace= True)
  SexConv = []
  for x in preprocess_df['Sex']:
    if x == 'male': 
      SexConv.append('1')
    elif x == 'female':
      SexConv.append('0')
  preprocess_df['SexConv'] = SexConv
  preprocess_df.drop(['Sex'], 1, inplace=True)
  return preprocess_df

In [18]:

processDF = preprocess(train)
print(processDF)

#Scaling data
sc = StandardScaler()
x = train.drop(['Survived'], 1)
y = train['Survived']
x = pd.DataFrame(sc.fit_transform(x.values), index=x.index, columns= x.columns)


     Survived  Pclass Age  SibSp  Parch SexConv
0           0       3  22      1      0       1
1           1       1  38      1      0       0
2           1       3  26      0      0       0
3           1       1  35      1      0       0
4           0       3  35      0      0       1
..        ...     ...  ..    ...    ...     ...
886         0       2  27      0      0       1
887         1       1  19      0      0       0
888         0       3  29      1      2       0
889         1       1  26      0      0       1
890         0       3  32      0      0       1

[891 rows x 6 columns]


In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [36]:
x_test

,Pclass,Age,SibSp,Parch,SexConv
709,0.827377,-0.043103,0.432793,0.767630,0.737695
439,-0.369365,0.110771,-0.474545,-0.473674,0.737695
840,0.827377,-0.735532,-0.474545,-0.473674,0.737695
720,-0.369365,-1.812645,-0.474545,0.767630,-1.355574
39,0.827377,-1.197152,0.432793,-0.473674,-1.355574
...,...,...,...,...,...
433,0.827377,-0.966342,-0.474545,-0.473674,0.737695
773,0.827377,-0.043103,-0.474545,-0.473674,0.737695
25,0.827377,0.649327,0.432793,5.732844,-1.355574
84,-0.369365,-0.966342,-0.474545,-0.473674,-1.355574


At first, let's try LogistRegression.

In [32]:
lg = LogisticRegression(solver= 'lbfgs', random_state=42)
lg.fit(x_train, y_train)
logistic_prediction = lg.predict(x_test)
score = accuracy_score(y_test, logistic_prediction)
#confusion_matrix(y_test, logistic_prediction, score=score)

f1 = f1_score(y_test, logistic_prediction)
print(score, f1)

0.8100558659217877 0.7671232876712328


Not so bad score! Now we are going to compare different linear models.
But first, let's confirm my thesis: do we really do not need of dropped columns?
